# N-body simulation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
def get_grav_acc(x, m, G):
    """
    Calculate gravitational acceleration of all points.
        x: (n, 2) array of positions
        m: (n,) array of masses
        G: gravitational constant
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    x = x.to(device)
    m = m.to(device)
    
    n = x.shape[0]
    assert x.shape == (n, 2) and m.shape == (n,)

    epsilon = 5  # For buffering/smoothing effect

    # Calculate pairwise displacement vectors (x_i - x_j)
    dx = x[:, None, :] - x[None, :, :]  # Shape: (n, n, 2)
    d = torch.norm(dx, dim=2)

    mapped_masses = m[:, None].expand(n, n)
    F = G * dx * mapped_masses[:, :, None] / (d**3 + epsilon)[:, :, None]
    acc = torch.sum(F, dim=0) / m[:, None]

    return acc

In [4]:
def update_system(x, v, m, dt, G):
    """
    Update points due to gravitational attraction.
        x:  (n, 2) array of positions
        v:  (n, 2) array of velocities
        m:  (n,) array of masses
        dt: time step
        G:  gravitational constant
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    x = x.to(device)
    v = v.to(device)
    m = m.to(device)
    
    n = x.shape[0]
    assert x.shape == (n, 2) and v.shape == (n, 2) and m.shape == (n,)
    
    # Update positions and velocities using Verlet integration
    # https://en.wikipedia.org/wiki/Verlet_integration#Velocity_Verlet
    a = get_grav_acc(x, m, G)

    x_new = x + v * dt + 0.5 * a * dt**2
    a_new = get_grav_acc(x_new, m, G)
    v_new = v + 0.5 * (a + a_new) * dt

    return x_new, v_new

In [ ]:
import imageio
import cv2

n = 1000
width = 512
height = 512
G = 1

theta = np.random.random((n,)) * (2 * np.pi)
r = (np.random.random((n,)) * 0.8 + 0.1) * (width/2)
x = np.array([np.cos(theta) * r + (width/2), np.sin(theta) * r + (height/2)]).T
v = np.zeros((n, 2))
m = np.ones((n,))

# Chonky boi
x[0] = [width/2, height/2]
m[0] = 10000
for i in range(1, n):
    dx = x[i] - x[0]
    dx = np.array([dx[1], -dx[0]])
    r = np.linalg.norm(dx)
    dx *= np.sqrt(G * m[0]) * (1/r) * r**(-1/2)
    v[i] = dx
v[0] = [0, 0]

x = torch.from_numpy(x)
v = torch.from_numpy(v)
m = torch.from_numpy(m)

In [8]:
frames = []
fig, ax = plt.subplots(figsize=(width / 100, height / 100), dpi=100)
fig.patch.set_facecolor('black')

for i in tqdm(range(30)):
    x, v = update_system(x, v, m, 0.1, G)
    ax.clear()
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    ax.scatter(x.cpu()[:,0], x.cpu()[:,1], color="white", s=torch.sqrt(m))
    plt.axis("off")
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.buffer_rgba(), dtype="uint8")
    image = image.reshape(fig.canvas.get_width_height() + (4,))
    image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
    
    # Make the image writable by creating a copy
    image = image.copy()
    frames.append(image)

plt.close()

# Save frames as an animated GIF with looping
imageio.mimsave("nbody_test.gif", frames, fps=30, loop=0)

100%|██████████| 30/30 [00:01<00:00, 27.35it/s]
